# Notebook : 01_fruits_pipeline_cloud.ipynb - Version corrigée
# Pipeline de traitement des images de fruits avec PySpark

# ============================================================================
# 📋 CELLULE 1 : IMPORTS ET CONFIGURATION
# ============================================================================

In [1]:
import os
import sys
import warnings

# === SUPPRESSION RADICALE DE TOUS LES WARNINGS ===
warnings.filterwarnings('ignore')

# Configuration AVANT tous les imports
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'

# Redirection temporaire de stderr pour masquer les messages TensorFlow/CUDA
original_stderr = sys.stderr
if os.name != 'nt':  # Linux/Mac
    sys.stderr = open('/dev/null', 'w')
else:  # Windows
    sys.stderr = open('nul', 'w')

try:
    from pyspark.sql import SparkSession
    from pyspark.sql import functions as F
    from pyspark.ml.feature import PCA
    import matplotlib.pyplot as plt
    import numpy as np
    from tensorflow.python.client import device_lib
    
finally:
    # Restauration de stderr après les imports
    sys.stderr.close() 
    sys.stderr = original_stderr

# Configuration matplotlib silencieuse
plt.rcParams.update({'figure.max_open_warning': 0})
import matplotlib
matplotlib.use('Agg')  # Mode non-interactif

# Préservation de la fonction sum() native Python
python_sum = __builtins__['sum'] if isinstance(__builtins__, dict) else __builtins__.sum

# Ajout du répertoire src au PYTHONPATH pour les imports
sys.path.append('../src')

# Imports des modules personnalisés
from preprocessing import load_images_from_directory, extract_features_mobilenet
from pca_reduction import convert_array_to_vector, get_optimal_pca_k, plot_variance_explained, apply_pca_on_features, plot_variance_curve
from utils import export_dataframe_if_needed, setup_project_directories, clean_gpu_cache

clean_gpu_cache()

print("✅ Imports réalisés avec succès")

[INFO] Mémoire GPU nettoyée avec succès.
✅ Imports réalisés avec succès


# ============================================================================
# 📋 CELLULE 2 : INITIALISATION SPARK
# ============================================================================
## 2.1 - Configuration Spark pour minimiser les logs

In [2]:
spark = SparkSession.builder \
    .appName("FruitsPipelineCloud") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sql.adaptive.advisoryPartitionSizeInBytes", "128MB") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/07/24 17:19:35 WARN Utils: Your hostname, PC-ARNAUD resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/24 17:19:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/24 17:19:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 2.2 - Configuration des logs pour réduire la verbosité au minimum

In [3]:
spark.sparkContext.setLogLevel("ERROR")

print(f"🚀 Session Spark créée : {spark.version}")
print(f"📊 Nombre de cœurs disponibles : {spark.sparkContext.defaultParallelism}")

🚀 Session Spark créée : 3.4.1
📊 Nombre de cœurs disponibles : 16


# ============================================================================
# 📋 CELLULE 3 : CONFIGURATION DES CHEMINS ET RÉPERTOIRES
# ============================================================================

## 3.1 - Configuration des chemins de travail

In [4]:
DATA_PATH = "../data/fruits-360/Test"
OUTPUTS_PATH = "../outputs"
CACHE_PATH = "../outputs/cache"

## 3.2 - Création automatique de l'arborescence du projet

In [5]:
directories = setup_project_directories(base_path="../")

print(f"📁 Chemin des données : {DATA_PATH}")
print(f"📁 Chemin de sortie : {OUTPUTS_PATH}")

📁 Tous les répertoires existent déjà
📁 Chemin des données : ../data/fruits-360/Test
📁 Chemin de sortie : ../outputs


## 3.3 - Vérification de l'existence des données

In [6]:
if not os.path.exists(DATA_PATH):
    print(f"❌ ERREUR : Le répertoire {DATA_PATH} n'existe pas !")
    print("💡 Assure-toi d'avoir téléchargé et extrait le dataset Fruits-360")
else:
    # Utilisation de la fonction sum() native Python (préservée en cellule 1)
    total_images = python_sum([len(files) for r, d, files in os.walk(DATA_PATH) if files])
    print(f"📸 Nombre total d'images détectées : {total_images}")

📸 Nombre total d'images détectées : 22688


# ============================================================================
# 📋 CELLULE 4 : CHARGEMENT DES DONNÉES
# ============================================================================

In [7]:
print("🔄 Étape 1/5 : Chargement des images depuis le répertoire...")

🔄 Étape 1/5 : Chargement des images depuis le répertoire...


## 4.1 - Chargement des chemins d'images et labels via la fonction externalisée

In [8]:
df_images = load_images_from_directory(
    spark=spark, 
    data_path=DATA_PATH,
    sample_size=500,  # Limitation pour les tests - à augmenter en production
    cache_path=f"{CACHE_PATH}/images_paths.parquet",
    force_retrain=False
)

print("✅ Chargement terminé")
print("📊 Aperçu des données :")
df_images.show(5, truncate=False)
print(f"📈 Nombre total d'images chargées : {df_images.count()}")

✅ Chargement depuis le cache : ../outputs/cache/images_paths.parquet


✅ Chargement terminé
📊 Aperçu des données :
+---------------------------------------------------+-------------+
|path                                               |label        |
+---------------------------------------------------+-------------+
|../data/fruits-360/Test/Grape White 3/169_100.jpg  |Grape White 3|
|../data/fruits-360/Test/Grape White 3/132_100.jpg  |Grape White 3|
|../data/fruits-360/Test/Grape White 3/r_153_100.jpg|Grape White 3|
|../data/fruits-360/Test/Grape White 3/147_100.jpg  |Grape White 3|
|../data/fruits-360/Test/Strawberry/r_52_100.jpg    |Strawberry   |
+---------------------------------------------------+-------------+
only showing top 5 rows

📈 Nombre total d'images chargées : 500


# ============================================================================
# 📋 CELLULE 5 : EXTRACTION DES FEATURES AVEC MOBILENETV2
# ============================================================================

In [9]:
print("\n🔄 Étape 2/5 : Extraction des features avec MobileNetV2...")
print("⚠️  Cette étape peut prendre plusieurs minutes selon le nombre d'images")


🔄 Étape 2/5 : Extraction des features avec MobileNetV2...
⚠️  Cette étape peut prendre plusieurs minutes selon le nombre d'images


## 5.1 - Extraction des caractéristiques via transfert learning - fonction externalisée

In [10]:
df_features = extract_features_mobilenet(
    spark = spark,
    df = df_images,
    cache_path = f"{CACHE_PATH}/features_mobilenet_gpu.parquet",
    force_retrain = False,
    batch_size = 32  # Optimisé pour GTX 1060 6GB
)

print("✅ Extraction des features terminée")
print("📊 Vérification des dimensions des features :")
print(device_lib.list_local_devices())

✅ Chargement depuis le cache : ../outputs/cache/features_mobilenet_gpu.parquet
✅ Extraction des features terminée
📊 Vérification des dimensions des features :
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8506407642985989002
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4807720960
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16688862185808198896
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:06:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


## 5.2 - Inspection d'un échantillon de features

In [11]:
sample_features = df_features.select("features").first()["features"]
print(f"🎯 Dimension des vecteurs de caractéristiques : {len(sample_features)}")
print(f"🎯 Type des données : {type(sample_features)}")
print(f"🎯 Exemple de valeurs : {sample_features[:10]}...")

🎯 Dimension des vecteurs de caractéristiques : 1280
🎯 Type des données : <class 'list'>
🎯 Exemple de valeurs : [0.690702497959137, 0.9102954268455505, 0.0, 0.0, 0.08946313709020615, 0.0, 0.3325121998786926, 0.7319117188453674, 0.1733175814151764, 0.023059630766510963]...


# ============================================================================
# 📋 CELLULE 6 : CONVERSION AU FORMAT SPARK ML
# ============================================================================

In [12]:
print("\n🔄 Étape 3/5 : Conversion des données au format Spark ML...")


🔄 Étape 3/5 : Conversion des données au format Spark ML...


### Conversion nécessaire pour PCA via fonction externalisée
## 6.1 - MobileNetV2 retourne des arrays Python, mais PCA Spark attend des VectorUDT

In [13]:
df_features_converted = convert_array_to_vector(df_features, features_col = "features")

print("✅ Conversion terminée")

🔄 Conversion des arrays Python vers des vecteurs Spark ML...
✅ Conversion terminée - les features sont maintenant au format VectorUDT
✅ Conversion terminée


## 6.2 - Vérification du nouveau format

In [14]:
print("📊 Vérification du schéma après conversion :")
df_features_converted.printSchema()

📊 Vérification du schéma après conversion :
root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- features: vector (nullable = true)



# ============================================================================
# 📋 CELLULE 7 : CALCUL DE LA VARIANCE CUMULÉE POUR DÉTERMINER K OPTIMAL
# ============================================================================

In [15]:
print("\n🔄 Étape 4/5 : Analyse de la variance pour déterminer le nombre optimal de composantes...")
print("📊 Recherche du nombre de composantes pour 95% de variance expliquée")


🔄 Étape 4/5 : Analyse de la variance pour déterminer le nombre optimal de composantes...
📊 Recherche du nombre de composantes pour 95% de variance expliquée


## 7.1 - Appel de la fonction externalisée pour calculer k optimal
- max_k augmenté à 200 pour MobileNetV2 (1280 features)

In [16]:
k_optimal, variance_data = get_optimal_pca_k(
    df=df_features_converted,
    spark=spark,
    max_k=200,
    threshold=0.95,
    force_retrain=False,
    cache_path=f"{CACHE_PATH}/pca_variance_analysis.parquet"
)

print(f"\n🎯 RÉSULTAT : k_optimal = {k_optimal} composantes")
print(device_lib.list_local_devices())

✅ Cache de variance chargé depuis ../outputs/cache/pca_variance_analysis.parquet

🧪 Aperçu des résultats :
+---+--------------------+--------------------+
|k  |individual_variance |cum_variance        |
+---+--------------------+--------------------+
|1  |0.2655833735146533  |0.2655833735146533  |
|2  |0.20852696638906548 |0.4741103399037188  |
|3  |0.11059546457472673 |0.5847058044784457  |
|4  |0.01242247940609405 |0.053790039499492494|
|5  |0.031028624082689037|0.6948871523099389  |
|6  |0.02876907008832725 |0.7236562223982663  |
|7  |0.02184833911639816 |0.7455045615146643  |
|8  |0.016204624921230506|0.7617091864358949  |
|9  |0.013424473540920011|0.7751336599768149  |
|10 |0.011709280162410884|0.7868429401392261  |
|11 |0.010219268569100324|0.7970622087083259  |
|12 |0.00919384975557515 |0.8062560584639012  |
|13 |0.010533789831949896|0.1552042951903053  |
|14 |0.008077442165124614|0.8232170192551493  |
|15 |0.007609229970823866|0.8308262492259728  |
|16 |0.006763631481942091|0.8

In [17]:
# Données (assure-toi que variance_data est bien défini)
ks = [row[0] for row in variance_data]
cum_vars = [row[2] for row in variance_data]

# Tracé
plt.figure(figsize=(10, 6))
plt.plot(ks, cum_vars, marker='o', linestyle='-', color='blue')
plt.axhline(y=0.95, color='red', linestyle='--', label='Seuil 95%')
plt.axvline(x=k_optimal, color='green', linestyle='--', label=f'k optimal = {k_optimal}')
plt.title("Variance cumulée en fonction de k (PCA)")
plt.xlabel("Nombre de composantes principales (k)")
plt.ylabel("Variance cumulée")
plt.grid(True)
plt.legend()
plt.tight_layout()

# Sauvegarde (à la place de plt.show())
plt.savefig("../outputs/pca_variance_plot.png")
print("📈 Graphique sauvegardé dans outputs/pca_variance_plot.png")

📈 Graphique sauvegardé dans outputs/pca_variance_plot.png


In [18]:
plot_variance_curve(variance_data, k_optimal, save_path=f"{OUTPUTS_PATH}/pca_variance_final.png")

# ============================================================================
# 📋 CELLULE 8 : GÉNÉRATION DU GRAPHIQUE EMPIRIQUE DE VARIANCE EXPLIQUÉE
# ============================================================================

In [19]:
print("\n📈 Génération du graphique de variance expliquée...")


📈 Génération du graphique de variance expliquée...


## 8.1 - Création du graphique empirique via fonction externalisée

In [20]:
plot_variance_explained(
    variance_data=variance_data,
    threshold=0.95,
    save_path=f"{OUTPUTS_PATH}/pca_variance_analysis.png"
)

print("✅ Graphique généré et sauvegardé")

📈 Génération du graphique de variance expliquée...
💾 Graphique sauvegardé : ../outputs/pca_variance_analysis.png
✅ Graphique généré et sauvegardé


# ============================================================================
# 📋 CELLULE 9 : APPLICATION DE L'ACP AVEC K OPTIMAL
# ============================================================================

In [21]:
print(f"\n🔄 Étape 5/5 : Application de l'ACP avec k={k_optimal} composantes...")


🔄 Étape 5/5 : Application de l'ACP avec k=56 composantes...


## 9.1 - Application de la réduction PCA avec le k optimal déterminé via fonction externalisée

In [22]:
df_pca_optimal = apply_pca_on_features(
    spark=spark,
    df=df_features_converted,
    k=k_optimal,
    features_col="features",
    output_path=f"{OUTPUTS_PATH}/features_pca_optimal.parquet",
    force_retrain=False
)

print("✅ ACP appliquée avec succès")
print(device_lib.list_local_devices())

✔ Chargement des données PCA depuis le cache : ../outputs/features_pca_optimal.parquet
✅ ACP appliquée avec succès
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9854933878986181916
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4807720960
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5688865627447631963
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:06:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


## 9.2 - Vérification des résultats

In [23]:
print("📊 Vérification des données après ACP :")
df_pca_optimal.printSchema()

📊 Vérification des données après ACP :
root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- features_pca: vector (nullable = true)



## 9.3 - Inspection des dimensions réduites

In [24]:
sample_pca = df_pca_optimal.select("features_pca").first()["features_pca"]
print(f"🎯 Dimensions après réduction : {sample_pca.size}")
print(f"🎯 Facteur de réduction : {1280 / sample_pca.size:.1f}x")

🎯 Dimensions après réduction : 56
🎯 Facteur de réduction : 22.9x


# ============================================================================
# 📋 CELLULE 10 : SAUVEGARDE ET VALIDATION FINALE
# ============================================================================

In [25]:
print("\n💾 Sauvegarde des résultats finaux...")


💾 Sauvegarde des résultats finaux...


## 10.1 - Sélection des colonnes finales pour la sauvegarde

In [26]:
df_final = df_pca_optimal.select("path", "label", "features_pca")

## 10.2 - Sauvegarde au format Parquet (optimal pour Spark)

In [27]:
final_parquet_path = f"{OUTPUTS_PATH}/final_results.parquet"
df_final.write.mode("overwrite").parquet(final_parquet_path)

## 10.3 - Sauvegarde au format CSV pour compatibilité

In [28]:
# Drop des colonnes non compatibles avec CSV
df_export = df_final.drop("features_pca")

# Sauvegarde CSV propre
final_csv_path = f"{OUTPUTS_PATH}/final_results.csv"
df_export.coalesce(1).write.mode("overwrite").option("header", "true").csv(final_csv_path)

print(f"✅ Résultats sauvegardés :")
print(f"   - Format Parquet : {final_parquet_path}")
print(f"   - Format CSV : {final_csv_path}")

✅ Résultats sauvegardés :
   - Format Parquet : ../outputs/final_results.parquet
   - Format CSV : ../outputs/final_results.csv


# ============================================================================
# 📋 CELLULE 11 : RÉSUMÉ ET STATISTIQUES FINALES
# ============================================================================

In [29]:
print("\n" + "="*60)
print("📊 RÉSUMÉ DU PIPELINE DE TRAITEMENT")
print("="*60)

print(f"🗂️  Nombre d'images traitées : {df_final.count()}")
print(f"🏷️  Nombre de classes détectées : {df_final.select('label').distinct().count()}")
print(f"📐 Dimensions originales (MobileNetV2) : 1280")
print(f"📐 Dimensions après ACP : {k_optimal}")
print(f"📊 Variance expliquée : 95%+")


📊 RÉSUMÉ DU PIPELINE DE TRAITEMENT
🗂️  Nombre d'images traitées : 500
🏷️  Nombre de classes détectées : 4
📐 Dimensions originales (MobileNetV2) : 1280
📐 Dimensions après ACP : 56
📊 Variance expliquée : 95%+


## 11.1 - Calcul de la taille du fichier final

In [30]:
if os.path.exists(final_parquet_path):
    file_size_mb = sum([os.path.getsize(os.path.join(final_parquet_path, f)) 
                       for f in os.listdir(final_parquet_path) 
                       if os.path.isfile(os.path.join(final_parquet_path, f))]) / (1024*1024)
    print(f"💾 Taille du fichier final : {file_size_mb:.1f} MB (Parquet)")

print("\n✅ PIPELINE TERMINÉ AVEC SUCCÈS !")
print("💡 Les données sont prêtes pour le déploiement cloud ou l'entraînement de modèles")

💾 Taille du fichier final : 0.3 MB (Parquet)

✅ PIPELINE TERMINÉ AVEC SUCCÈS !
💡 Les données sont prêtes pour le déploiement cloud ou l'entraînement de modèles


## 11.2 - Arrêt propre de la session Spark

In [31]:
# spark.stop()
# print("🔴 Session Spark fermée")